In [33]:
import os 

In [1]:
from collections import defaultdict

In [35]:
output_root = './UDN_freqeuncy/'

In [2]:
gtfileame = './extractUDN/all_groundtruth.txt'
seqfilename = './extractUDN/all_input.txt'

### Extract chararcter frequency

In [12]:
char_freq = dict()
with open('UDN_charCount.txt', 'r', encoding='utf8') as fp:
#     print(fp.readline())
    for line in fp:
        lst = line.strip().split(',')
        char_freq[lst[0]] = int(lst[-1])

### Extract error-correct paris

In [3]:
bigDict = defaultdict(lambda: defaultdict(int))
combine = defaultdict(lambda :[0,0])

with open(gtfileame, 'r', encoding='utf8') as gtp,\
    open(seqfilename, 'r', encoding='utf8') as ip:
    line_idx = 0
    seq_line = ip.readline().strip('\n')
    dataID, seq = seq_line[:seq_line.find(',')], seq_line[seq_line.find(',')+2:]

    gt_lst  = gtp.readline().strip('\n').split(', ')
    while seq!='' or gt_lst!=['']:
        gt_lst  = gtp.readline().strip('\n').split(', ')

        seq_line = ip.readline().strip('\n')
        dataID, seq = seq_line[:seq_line.find(',')], seq_line[seq_line.find(',')+2:]

        # Avoid some exception of sequence that have unwanted \M
        cntdown = 5
        while dataID != gt_lst[0] and cntdown!=0:
            seq_line = ip.readline().strip('\n')
            dataID, seq = seq_line[:seq_line.find(',')], seq_line[seq_line.find(',')+2:]
            cntdown -= 1
        # If the count of trial above, jump out for bad input data 
        if cntdown==0:
            break

        line_idx += 1    

        gt = gt_lst[1:]    
        for idx in range(0, len(gt), 2):
            corr_ch = str(gt[idx+1])
            try:
                error_ch = str(seq[int(gt[idx])-1])
            except IndexError:
                continue
                
            if error_ch>corr_ch:
                combine[(error_ch, corr_ch)][0] += 1
            else:
                combine[(corr_ch, error_ch)][1] += 1

            bigDict[error_ch][corr_ch] += 1

    print('Done.')
    print('= Processing {} sequences'.format(line_idx))
    print('= Total Error character {}'.format(len(bigDict)))

Done.
= Processing 54624 sequences
= Total Error character 2653


## Data what we have
1. combine[pair] = [cnt1, cnt2]
2. char_freq[ch] = [cnt]

### Most common case 

In [75]:
mostCommonCase_pair = sorted(combine.items(), key= lambda x:max(x[1]), reverse=True)

In [80]:
mostCommonCase_char = []
for pair, counts in mostCommonCase_pair:
    try:
        mostCommonCase_char.append(
                    (pair[0],pair[1],counts[0],char_freq[pair[0]], char_freq[pair[1]]))
        mostCommonCase_char.append(
            (pair[1],pair[0],counts[1],char_freq[pair[1]], char_freq[pair[0]]))
    except KeyError:
        continue

In [81]:
mostCommonCase_char = sorted(mostCommonCase_char, key=lambda x:-x[2])

In [83]:
with open(os.path.join(output_root, 'mostCommonCase_withDigit.csv'), 'w', encoding='utf8') as wp:
    wp.write('Error,Correct,Count,CountOfError,CountOfCorrect\n')
    for item in mostCommonCase:
        wp.write('{p[0]},{p[1]},{p[2]},{p[3]},{p[4]}\n'.format(p=item))

#### Kick number

In [84]:
mostCommonCase_char_nodigit = []
for item in mostCommonCase_char:
    if item[0].isdigit() or item[1].isdigit():
        pass
    else:
        mostCommonCase_char_nodigit.append(item)

In [88]:
with open(os.path.join(output_root, 'mostCommonCase_nodigit.csv'), 'w', encoding='utf8') as wp:
    wp.write('Error,correct,count,ErFreq,CorFreq\n')
    for item in mostCommonCase_char_nodigit:
        wp.write('{item[0]},{item[1]},{item[2]},{item[3]},{item[4]}\n'.format(item=item))

In [90]:
mostCommonCase_pair_nodigit = []
for pair, counts in mostCommonCase_pair:
    if pair[0].isdigit() or pair[1].isdigit():
        pass
    else:
        mostCommonCase_pair_nodigit.append((pair, counts))

### Number transform case

In [45]:
mostCommonCase_pair[65]

(('３', '3'), [63, 1])

In [59]:
numberTransform = []
for pair, counts in mostCommonCase_pair:
    if pair[0].isdigit() or pair[1].isdigit():
        try:
            numberTransform.append((pair,counts,[char_freq[pair[0]], char_freq[pair[1]]]))
        except KeyError:
            continue

In [62]:
with open(os.path.join(output_root, 'numberTransform.csv'), 'w', encoding='utf8') as wp:
    wp.write('P1,P2,p1->_p2,p2->p1,Fp1,Fp2\n')
    for pair, counts, frequency in numberTransform:
        wp.write('{p[0]},{p[1]},{c[0]},{c[1]},{f[0]},{f[1]}\n'.format(p=pair, c=counts, f=frequency))

In [53]:
numbermatch = {
    0:'零',
    1:'一',
    2:{'二','兩'},
    3:'三',
    4:'四',
    5:'五',
    6:'六',
    7:'七',
    8:'八',
    9:'九'
}

### Mutual case (error->correct) and (correct->error)

In [103]:
abs(3-5)

2

In [108]:
def select(x, method):
    if method == 'sum':
        if x[1][0]==0 or x[1][1]==0:
            return 0
        else:
            return sum(x[1])
    elif method == 'differ':
        if x[1][0]==0 or x[1][1]==0:
            return 0
        else:
            return -abs(x[1][0]-x[1][1])
    
mostCommoMutalCase = sorted(mostCommonCase_pair_nodigit, key=lambda x:select(x,'sum'),reverse=True)

In [102]:
with open(os.path.join(output_root, 'mostCommonMutualCase_sum.csv'), 'w', encoding='utf8') as wp:
    wp.write('p1,p2,p1->p2,p2->p1,Fp1,Fp2\n')
    for pair, counts in mostCommoMutalCase:
        try:
            freq_pair = [char_freq[pair[0]], char_freq[pair[1]]]
        except KeyError:
            continue
        wp.write('{p[0]},{p[1]},{c[0]},{c[1]},{fpF[0]},{fpF[1]}\n'.
                 format(p=pair,c=counts,fpF = freq_pair))

In [112]:
mostCommonCase_pair_nodigit_MutualOnly = [(pair, counts)
    for pair, counts in mostCommonCase_pair_nodigit if counts[0] != 0 and counts[1] != 0]

In [114]:
mostCommoMutalCaseMean = sorted(mostCommonCase_pair_nodigit_MutualOnly, key=lambda x:select(x,'differ'), reverse=True)

In [121]:
with open(os.path.join(output_root, 'mostCommonMutualCase_diff.csv'), 'w', encoding='utf8') as wp:
    wp.write('p1,p2,p1->p2,p2->p1,Fp1,Fp2\n')
    for pair, counts in mostCommoMutalCaseMean:
        try:
            freq_pair = [char_freq[pair[0]], char_freq[pair[1]]]
        except KeyError:
            continue
        wp.write('{p[0]},{p[1]},{c[0]},{c[1]},{fpF[0]},{fpF[1]}\n'.
                 format(p=pair,c=counts,fpF = freq_pair))

### Most transform mutual case 